In [1]:
import sys
from nbfinder import NotebookFinder
sys.meta_path.append(NotebookFinder())
import numpy as np
import tensorflow as tf
if __name__ == "__main__":
    sys.path.append("../../")
from make_anchors_orig import make_anchors
from notebooks_src.configs import configs
from notebooks_src.box_processing.np_util import make_box_coords_relative, extract_box_coordinates
from notebooks_src.box_processing.tf_box_util import ious_with_anchors, convert_to_xyminmax,\
encode_to_scaled_offsets, convert_to_yxhw,make_actual_gt_box_mask
import numpy as np
from make_anchors_orig import make_anchors


importing Jupyter notebook from make_anchors_orig.ipynb
importing Jupyter notebook from ../../notebooks_src/configs/box_encode_decode_configs.ipynb
importing Jupyter notebook from ../../notebooks_src/configs/tensorboard_configs.ipynb
importing Jupyter notebook from ../../notebooks_src/configs/fit_configs.ipynb
importing Jupyter notebook from ../../notebooks_src/configs/labels_configs.ipynb
importing Jupyter notebook from ../../notebooks_src/configs/load_data_configs.ipynb
importing Jupyter notebook from ../../notebooks_src/configs/losses_configs.ipynb
importing Jupyter notebook from ../../notebooks_src/configs/metrics_configs.ipynb
importing Jupyter notebook from ../../notebooks_src/configs/models_configs.ipynb
importing Jupyter notebook from ../../notebooks_src/configs/optimizers_configs.ipynb
importing Jupyter notebook from ../../notebooks_src/box_processing/np_util.ipynb
importing Jupyter notebook from ../../notebooks_src/box_processing/tf_box_util.ipynb


In [53]:
def encode(bboxes, labels):
    labels = np.where(labels==-1.,0.,labels)
    bboxes = np.where(bboxes==-1.,0.,bboxes)
    bboxes = make_box_coords_relative(bboxes)
    ymin, ymax, xmin,xmax = extract_box_coordinates(bboxes)
    actual_gt_box_mask = make_actual_gt_box_mask(ymin, ymax, xmin, xmax)
    all_anchors = make_anchors()
    #get the shape of each anchor array -> same as fmap shapes
    fmap_shapes = [anchor[0].shape[:2] for anchor in all_anchors]
    #dictionary mapping fmap shape to anchors for that fmap shape
    anchors_map = dict(zip(fmap_shapes,all_anchors))
    
    encoded_boxes_dict = {}
    encoded_labels_dict = {}
    for fmap_shape, anchors in anchors_map.iteritems():
        encoded_boxes_dict[fmap_shape] = _encode_boxes(bboxes, anchors)
        encoded_labels_dict[fmap_shape] = _encode_labels(labels, anchors)
        
    return encoded_boxes_dict, encoded_labels_dict
        
        
        
        
    
    
    

In [86]:
def _encode_labels(labels, anchors):
    ay, ax, ah, aw = anchors
    print (ay*ah).shape
    encoded_labels = labels * tf.expand_dims(tf.expand_dims(tf.ones_like(ay*ah),axis=-1),axis=-1)
    encoded_labels = tf.transpose(encoded_labels,perm=[3,0,1,2,4])
    return encoded_labels
   # tf.transpose(encoded_labels,perm=[])

In [87]:
def expand_anchors(anchors):
    ay, ax, ah, aw = anchors
    ay, ax = [np.expand_dims(tens, axis=-1) for tens in [ay,ax]]
    return ay,ax,ah,aw
    

In [89]:
def _encode_boxes(bboxes, anchors):
    ymin, ymax, xmin,xmax = extract_box_coordinates(bboxes)
    cy,cx,h,w = convert_to_yxhw(ymin, ymax,xmin, xmax)
    ay, ax, ah, aw = expand_anchors(anchors)
    enc_y, enc_x, enc_h, enc_w = encode_to_scaled_offsets(cy,cx,h,w, ay, ax, ah, aw)
    encoded_boxes = tf.stack([enc_y, enc_x, enc_h, enc_w],axis=-1)
    encoded_boxes = tf.transpose(encoded_boxes,perm=(2,0,1,4,5,3))
    return encoded_boxes
    
    

In [90]:
if __name__ == "__main__":
    import h5py
    bboxes = h5py.File(configs["tr_data_file"])["boxes"][7:12]
    print bboxes.shape
    bbox_coords = bboxes[:,:,:4]
    labels = bboxes[:,:,4]
    btens, ltens = encode(bbox_coords, labels)
    
    with tf.Session() as sess:
        tens = sess.run(ltens)

(5, 15, 5)
(6, 9, 6)
(48, 72, 6)
(24, 36, 6)
(12, 18, 6)
(96, 144, 4)
(1, 1, 4)
(3, 5, 4)
